# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [205]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [206]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [207]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [208]:
class MyDecisionTreeClassifier_original(BaseEstimator):
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, 
                 min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise Exception

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise Exception
            
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return ((1 - ((l_c / l_s) ** 2).sum(axis=1)) * l_s.T + (1 - ((r_c / r_s) ** 2).sum(axis=1)) * r_s.T)[0]
    
    def __entropy(self, l_c, l_s, r_c, r_s): 
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return ((-((l_c / l_s) * np.log(np.where(l_c / l_s == 0, 1, l_c / l_s))).sum(axis=1)) * l_s.T - \
                (((r_c / r_s) * np.log(np.where(r_c / r_s == 0, 1, r_c / r_s))).sum(axis=1)) * r_s.T)[0]

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return ((1 - np.max(l_c, axis=1) / l_s) * l_s.T + (1 - np.max(r_c, axis=1) / r_s) * r_s.T)[0]

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def _sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def _div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def _find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортирует параметр x и использует полученный индекс для y,
        # чтобы сохранить соответствие между значением параметра и целевой переменной.
        # class_number - число различных классов, которым принадлежат элементы в узле
        sorted_x, sorted_y = self._sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # Поскольку в листьях не может быть меньше min_samples_split элементов, мы можем делить только по индексам
        # в splitted_sorted_y. В r_border_ids вычисляется индекс, где следующий класс не равен предыдущему и затем смещается,
        # чтобы получить индексы, соответствующие возможным разбиениям в sorted_y, причем они указывают на первый элемент 
        # отличного класса.
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None 
        
        # Что делает этот блок кода?
        # eq_el_count -- то, сколько элеметов принадлежит каждому следующему классу
        # one_hot_code -- в каждой строчке записано то, какой класс будет крайним слева, если разделить по соответствущему 
        # id в r_borders_id
        # class increments -- то же самое, но кроме id класса в каждой строчке можно понять, сколько 
        # элементов принадлежит каждой группе подряд идущих элементов одного класса.
        # Причем в первой строчке также учитываются те элементы, которые есть в первых min_samples_split элементах
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split],
                                                                minlength=class_number)
        
        # Что делает этот блок кода?
        # l_class_count -- в каждой строчке написано, сколько элементов каждого класса будет в левом узле, если разбить
        # по соответствующему элементу
        # r_class_count -- то, сколько в правом узле
        # l_sizes -- размеры левого узла при разбиениях
        # r_sizes -- размеры правого
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Подсчитывается ошибка при каждом разбиении и берется то, при котором ошибка минимальна
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        
        # Что делает этот блок кода?
        # left_el_id -- указывает на крайний элемент правого узла
        # Функция возращает ошибку, которую даст лучшее разбиение и порог, по которому делится узел.
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id] + sorted_x[left_el_id + 1]) / 2.0

    def _fit_node(self, x, y, node_id, depth, pred_f=-1):
        leaf = [
            self.__class__.LEAF_TYPE, 
            np.argmax(np.bincount(y, minlength=self.num_class)), 
            np.bincount(y, minlength=self.num_class) * 1.0 / y.shape[0]
        ]
        
        if depth == self.max_depth:
            self.tree[node_id] = leaf
            return
        
        feature_ids = self.get_feature_ids(x.shape[1])
        
        split_data = []
        for feature in feature_ids:
            error, threshold = self._find_threshold(x[:, feature], y)
            split_data.append([feature, error, threshold])
        best_split = min(split_data, key=lambda split: split[1])
        x_l, x_r, y_l, y_r = self._div_samples(x, y, best_split[0], best_split[2])
        if best_split[1] == np.inf or not all([i.shape[0] for i in [y_l, y_r]]):
            self.tree[node_id] = leaf
            return
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, best_split[0], best_split[2])
        self._fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self._fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.tree = {}
        self.num_class = np.unique(y).size
        self._fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [209]:
def quality(model, quality_measure=accuracy_score):
    time0 = time()
    model.fit(x, y)
    print "Train time:\t{}".format(time() - time0)
    print "Train score:\t{}".format(quality_measure(y, model.predict(x)))
    
    gkf = KFold(n_splits=5, shuffle=True)
    scores = []
    for train, test in gkf.split(x, y):
        X_train, y_train = x[train], y[train]
        X_test, y_test = x[test], y[test]
        model.fit(X_train, y_train)
        scores.append(quality_measure(y_test, model.predict(X_test)))
    
    print "Avg test score:\t{}".format(sum(scores) / len(scores))

Проверим качество получившейся модели:

In [210]:
quality(MyDecisionTreeClassifier_original(min_samples_split=2))

Train time:	0.427999973297
Train score:	0.935120438351
Avg test score:	0.931786265794


Сравним с моделью из sklearn:

In [211]:
quality(DecisionTreeClassifier(min_samples_split=2))

Train time:	1.15300011635
Train score:	0.999983370611
Avg test score:	0.892075266314


Получилось, что наша модель обучается в 2 раза быстрее модели из sklearn, хуже обучается на данных, и при этом лучше предсказывает. Что здесь происходит?

Сначала посмотрим на эти 2 строчки:

```python
splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
```

В каждом листе будет не менее min_samples_split + 1 элементов, то есть то, что в коде называется min_samples_split является min_samples_leaf - 1 в sklearn. То есть модель, эквивалентная 
```python
MyDecisionTreeClassifier_original(min_samples_split=2)
```
будет 
```python
DecisionTreeClassifier(min_samples_leaf=3)
```
Посмотрим на разницу:

In [212]:
quality(DecisionTreeClassifier(min_samples_leaf=3))

Train time:	0.930000066757
Train score:	0.968354272506
Avg test score:	0.903690921787


Стало немного больше похоже на нашу модель, но разница все еще велика.

Посмотрим на дерево, которое строит наша модель, а точнее на самый первый узел:

In [213]:
clf_original = MyDecisionTreeClassifier_original(min_samples_split=2)
clf_original.fit(x, y)
clf_original.tree[0]

(0, 6, 1.0)

Что будет, если мы поделим по 6-ой фиче с порогом 1?

In [214]:
x_l, x_r, y_l, y_r = clf_original._div_samples(x, y, 6, 1.0)
y_l.shape, y_r.shape, y.shape

((2413L,), (117856L,), (120269L,))

117856 из 120269 элементов будут отнесены к правому узлу. А правый узел ...

In [215]:
clf_original.tree[0 * 2 + 2]

[1, 0, array([ 0.94046124,  0.05953876])]

...является листом. То есть 98% всех элементов оказались в одном листе. Это объясняет скорость алгоритма, но как при этом получается приличное качество? Посмотрим на наши данные:

In [216]:
pd.Series(y).value_counts()

0    111912
1      8357
dtype: int64

В данных сильное смещение в сторону класса 0, и даже простое предсказание 'все нули' дает хорошее качество:

In [217]:
accuracy_score(np.zeros(y.shape), y)

0.930514097564626

Это ожидаемо, ведь accuracy_score плохо подходит для не сбалансированных классов. Лучше использовать f-measure или roc_auc:

In [218]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

print 'Our model\n\nF-measure'
quality(MyDecisionTreeClassifier_original(min_samples_split=2), f1_score)
print '\nROC-AUC'
quality(MyDecisionTreeClassifier_original(min_samples_split=2), roc_auc_score)

Our model

F-measure
Train time:	0.431999921799
Train score:	0.187779743937
Avg test score:	0.151378463565

ROC-AUC
Train time:	0.427999973297
Train score:	0.552411941422
Avg test score:	0.549941893078


In [219]:
print 'sklearn model\n\nF-measure:'
quality(DecisionTreeClassifier(min_samples_leaf=3), f1_score)
print '\nROC-AUC:'
quality(DecisionTreeClassifier(min_samples_leaf=3), roc_auc_score)

sklearn model

F-measure:
Train time:	0.880000114441
Train score:	0.751790597734
Avg test score:	0.272923686327

ROC-AUC:
Train time:	0.917999982834
Train score:	0.838832174057
Avg test score:	0.603788054989


Здесь уже видна сильная разница в качестве. Так почему качество нашей модели намного хуже? Почему она отнесла 98% всех элементов в один лист? Все дело в этой строчке:
```python
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
```
Дело в том, что мы хотим делить данные в местах, где меняется класс. Это правда, что если делить в местах смены класса ошибка будет минимальна, но ведь мы не проверили, что там можно делить. То есть может не существовать порога, который делит данные в том месте, где мы нашли минимальную ошибку. В частности алгоритм попытался разделить данные по 6-ой фиче и одному и тому же порогу два раза подряд, из-за чего и получось несбалансированное дерево.
Напишем исправленный класс:

In [220]:
class MyDecisionTreeClassifier(MyDecisionTreeClassifier_original):
    
    def __init__(self, **kwargs):
        self.min_samples_leaf = kwargs.pop('min_samples_leaf', 1)
        super(MyDecisionTreeClassifier, self).__init__(**kwargs)
        criterion_dict = {
            'gini': self.__gini,
            'entropy': self.__entropy,
            'misclass': self.__misclass
        }
        self.G_function = criterion_dict.get(kwargs.get('criterion', 'gini'))
        if self.G_function is None:
            raise Exception('invalid criterion name')
            
    def __gini(self, l_c, l_s, r_c, r_s):
        return (1 - ((l_c.T / l_s) ** 2).sum(axis=0)) * l_s + (1 - ((r_c.T / r_s) ** 2).sum(axis=0)) * r_s
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_c_T = l_c.T
        r_c_T = r_c.T
        return (-((l_c_T / l_s) * np.log(np.where(l_c_T == 0, 1, l_c_T / l_s))).sum(axis=0)) * l_s - \
                (((r_c_T / r_s) * np.log(np.where(r_c_T == 0, 1, r_c_T / r_s))).sum(axis=0)) * r_s

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (1 - np.max(l_c, axis=1) / l_s) * l_s + (1 - np.max(r_c, axis=1) / r_s) * r_s
    
    # Теперь x -- двумерный массив
    def _find_threshold(self, x, y):
        class_number = np.unique(y).shape[0]
        if class_number == 1:
            return None, float('+inf')
        
        n_samples = y.shape[0]
        x_args = np.argsort(x, axis=0)
        sorted_x = x[x_args, np.arange(x.shape[1])[np.newaxis, :]]
        sorted_y = y[x_args]
        borders = ((np.arange(n_samples - 1) >= self.min_samples_leaf - 1) & \
                  (np.arange(n_samples - 1) <= n_samples - self.min_samples_leaf)).reshape((1, -1))
        r_border_ids = (sorted_x[1:] != sorted_x[:-1]) & borders.T
        if r_border_ids.sum() == 0:
            return None, float('+inf')
        
        def classes_to_quality(arr):
            arr = arr.astype(int)
            one_hot = np.eye(class_number)[arr]
            l_c = one_hot.cumsum(axis=0)
            r_c = (l_c[-1] - l_c)[:-1]
            l_c = l_c[:-1]

            l_s = np.sum(l_c, axis=1)
            r_s = arr.shape[0] - l_s
            return self.G_function(l_c, l_s, r_c, r_s)
    
        quality = np.apply_along_axis(classes_to_quality, 0, sorted_y)
        left_el_idx, feature = np.unravel_index(np.where(r_border_ids, quality, np.inf).argmin(), x.shape)
        
        # берем не простое среднее, а с весами, пропорциональными размеру узлов
        return feature, sorted_x[left_el_idx, feature] * (left_el_idx + 1) / n_samples + \
               sorted_x[left_el_idx + 1, feature] * (1. - 1. * (left_el_idx + 1) / n_samples)

    def _fit_node(self, x, y, node_id, depth, pred_f=-1):
        leaf = [
            self.__class__.LEAF_TYPE, 
            np.argmax(np.bincount(y, minlength=self.num_class)), 
            np.bincount(y, minlength=self.num_class) * 1.0 / y.shape[0]
        ]
        
        if depth == self.max_depth or y.shape[0] < self.min_samples_split:
            self.tree[node_id] = leaf
            return
        
        feature_ids = self.get_feature_ids(x.shape[1])
        feature, threshold = self._find_threshold(x[:, feature_ids], y)
        if feature is None:
            self.tree[node_id] = leaf
            return
        
        x_l, x_r, y_l, y_r = self._div_samples(x, y, feature, threshold)
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, feature, threshold)
        self._fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self._fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
    

In [221]:
model = MyDecisionTreeClassifier(min_samples_leaf=1)
clf = DecisionTreeClassifier(min_samples_leaf=1)
model.fit(x, y)
clf.fit(x, y)

(model.predict(x) != clf.predict(x)).sum()

0

Исправленная модель обучилась на выборке точно также, как и модель из sklearn. Посмотрим на ее качество и скорость:

In [222]:
# original
quality(MyDecisionTreeClassifier_original(min_samples_split=2), f1_score)

Train time:	0.411999940872
Train score:	0.187779743937
Avg test score:	0.170461975606


In [223]:
# new, same params
quality(MyDecisionTreeClassifier(min_samples_leaf=3), f1_score)

Train time:	8.38400006294
Train score:	0.781754574812
Avg test score:	0.261732823104


In [224]:
# new, fit until same class in each leaf
quality(MyDecisionTreeClassifier(min_samples_leaf=1), f1_score)

Train time:	9.5720000267
Train score:	0.999880339835
Avg test score:	0.252502196653


In [225]:
# sklearn model, fit until same class in each leaf
quality(DecisionTreeClassifier(min_samples_leaf=1), f1_score)

Train time:	0.977999925613
Train score:	0.999880339835
Avg test score:	0.262430656716


Модель обучается теперь в 10 раз дольше, но и обучается, и предсказывает гораздо лучше. Вообще создавая модели с такими параметрами мы сильно переобучаем модель. По идее если останавливать обучение пораньше, скорость обучения и качество на тесте должны улучшиться:

In [226]:
# original
quality(MyDecisionTreeClassifier_original(min_samples_split=100), f1_score)

Train time:	0.238999843597
Train score:	0.1938558248
Avg test score:	0.199660454028


In [227]:
# new
quality(MyDecisionTreeClassifier2(min_samples_leaf=100), f1_score)

Train time:	2.44400000572
Train score:	0.298175907531
Avg test score:	0.263360772858


In [228]:
# sklearn model
quality(DecisionTreeClassifier(min_samples_leaf=100), f1_score)

Train time:	0.634999990463
Train score:	0.298148984199
Avg test score:	0.263020755368


До скорости sklearn еще далеко, но зато качество теперь примерно такое же.

Из экспериментов стало заметно, что _fit_threshold для 2-мерного массива x работает быстрее на маленьких выборок, а изначальный _fit_threshold для одномерного x -- на больших. Давайте воспользуемся этим и напишем класс, который использует один метод для больших x, и другой для маленьких:

In [229]:
class MyDecisionTreeClassifier_v2(MyDecisionTreeClassifier):
    
    # одномерный x
    def _find_threshold_big(self, x, y):
        sorted_x, sorted_y = self._sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        n_samples = y.shape[0]
        
        borders = ((np.arange(n_samples - 1) >= self.min_samples_leaf - 1) & \
                  (np.arange(n_samples - 1) <= n_samples - self.min_samples_leaf))
        x_borders = (sorted_x[1:] != sorted_x[:-1]) & borders
        y_borders = (sorted_y[1:] != sorted_y[:-1]) & borders
        
        #проверяем, можно ли использовать y_borders для деления
        if (y_borders == (x_borders & y_borders)).all():
            r_border_ids = y_borders
        else:
            r_border_ids = x_borders

        if ((r_border_ids).sum() == 0) or (class_number == 1):
            return float('+inf'), None
        
        one_hot_code = np.eye(class_number)[sorted_y]
        l_class_count = np.cumsum(one_hot_code, axis=0)[:-1][r_border_ids]
        r_class_count = np.bincount(sorted_y, minlength=class_number) - l_class_count
        l_sizes = l_class_count.sum(axis=1)
        r_sizes = n_samples - l_sizes

        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        left_el_id = int(l_sizes[idx])
        
        if sorted_x[left_el_id - 1] == sorted_x[left_el_id]:
            raise Exception
        return gs[idx], sorted_x[left_el_id - 1] * left_el_id / sorted_y.shape[0] + \
               sorted_x[left_el_id] * (1. - 1. * left_el_id / sorted_y.shape[0])


    def _fit_node(self, x, y, node_id, depth, pred_f=-1):
        n_samples = y.shape[0]
        leaf = [
            self.__class__.LEAF_TYPE, 
            np.argmax(np.bincount(y, minlength=self.num_class)), 
            np.bincount(y, minlength=self.num_class) * 1.0 / n_samples
        ]
        
        if depth == self.max_depth or n_samples < self.min_samples_split:
            self.tree[node_id] = leaf
            return
        
        feature_ids = self.get_feature_ids(x.shape[1])
        if n_samples < 1000:
            feature, threshold = self._find_threshold(x[:, feature_ids], y)    
        else:
            split_data = []
            for feature in feature_ids:
                error, threshold = self._find_threshold_big(x[:, feature], y)
                split_data.append([feature, error, threshold])
            feature, _, threshold = min(split_data, key=lambda split: split[1])
            
        if feature is None or threshold is None:
            self.tree[node_id] = leaf
            return
        
        x_l, x_r, y_l, y_r = self._div_samples(x, y, feature, threshold)
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, feature, threshold)
        self._fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self._fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
    

Посмотрим, какой выигрыш мы получили по сравнению с предыдущей моделью:

In [230]:
# предыдущая
quality(MyDecisionTreeClassifier(min_samples_leaf=1), f1_score)

Train time:	9.32099986076
Train score:	0.999880339835
Avg test score:	0.257549439563


In [231]:
# новая
quality(MyDecisionTreeClassifier_v2(min_samples_leaf=1), f1_score)

Train time:	7.98300004005
Train score:	0.999880339835
Avg test score:	0.256834378533


In [232]:
# предыдущая
quality(MyDecisionTreeClassifier(min_samples_leaf=100), f1_score)

Train time:	3.51900005341
Train score:	0.298175907531
Avg test score:	0.26153526087


In [233]:
# новая
quality(MyDecisionTreeClassifier_v2(min_samples_leaf=100), f1_score)

Train time:	2.33299994469
Train score:	0.298175907531
Avg test score:	0.262700334503


Выигрыш по времени при полном обучении : 10%, при неполном: 30%